# Plotting asynchronous steady-state EAs

This shows how to plot a run of an asynchronous steady-stae evolutionary algorithm (ASAE).

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
import sys
import itertools
import uuid

from leap import core
from leap import ops
from leap import binary_problems
from leap import real_problems

from leap.parallel.parallel import Parallel

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
class MyIndividual(core.Individual):
    """
        Adding `is_viable`.
    """
    birth = itertools.count()
    
    def __init__(self, genome, decoder=None, problem=None):
        super().__init__(genome, decoder, problem)

        # Used to uniquely identify this individual
        self.uuid = uuid.uuid4()
        
        # Track the current birth number
        self.birth = next(MyIndividual.birth)

    def __repr__(self):
        return " ".join([str(self.uuid), str(self.birth), str(self.fitness),
                         "".join([str(x) for x in self.genome])])

    def is_viable(self):
        """ This is used by Parallel to ensure that we are considering "viable"
        individuals.

        That is, an individual may have been returned from a worker as *not*
        viable because its evaluation was interrupted by, say, doing a check-
        point. In which case, we do not want to insert it into the pool.
        """
        return True

In [5]:
cluster = LocalCluster() # set up to run on local machine
client = Client(cluster)

In [6]:
my_parallel = Parallel(client, max_births=100, pool_size=10)

In [7]:
final_pop = my_parallel.do(MyIndividual, 
                           initializer=core.create_binary_sequence(11), 
                           init_pop_size=10, 
                           problem=binary_problems.MaxOnes(),
                           decoder=core.IdentityDecoder())

In [8]:
final_pop

[4f2fd76d-03ee-408c-b058-db04aba589d6 51 10 10111111111,
 00a2aef4-62ed-4187-b487-04be7a37b40f 55 10 10111111111,
 d8b51270-2a1a-4880-8d94-fe8e2355c1c0 36 11 11111111111,
 fa655e14-6214-40ec-9e50-13f092c0d6f6 20 10 10111111111,
 02ff6164-f9e7-4d45-a8a0-c2640e1f6031 24 10 10111111111,
 a2da69fd-41e2-44a2-8149-f9243e8315d4 39 11 11111111111,
 86ebb520-e465-4853-82e8-8dff92ce788b 48 11 11111111111,
 0e05a9a4-f7c5-480b-86ce-e033d4b373fb 2 10 10111111111,
 2bd5c88c-55f7-4532-a7cb-d68479f506a6 30 10 11011111111,
 92b300ef-87ec-4db6-bfcf-9dfa1d8048b0 56 10 11111101111]